In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import csv
import joblib
from sklearn.linear_model import LinearRegression
import ast
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# Initialize PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=True)

# Read CSV data
with open('calib_10xPonto/calib6/calibration10x_file17.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

In [ ]:
#TREINO RANDOM FOREST REGRESSOR
import numpy as np
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import csv

def load_and_convert_csv(file_path):
    data = []
    with open(file_path, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            # Converte cada valor na linha para float
            try:
                converted_row = [float(value) for value in row]
                data.append(converted_row)
            except ValueError as e:
                print(f"Aviso: Não foi possível converter a linha {row}: {e}")
                continue
    return np.array(data)  

# Exemplo de uso:
file_path = 'calib_10xPonto/calib10/calib_all_file10.csv'
data = load_and_convert_csv(file_path)


# Separando features (X) e targets (y)
features = data[:, :2]  # Primeiras duas colunas
target_X = data[:, 2]   # Terceira coluna (coordenada X)
target_Y = data[:, 3]    # Quarta coluna (coordenada Y)

# Dividindo em conjuntos de treino e teste (80% treino, 20% teste)
X_train, X_test, y_X_train, y_X_test = train_test_split(features, target_X, test_size=0.2, random_state=42)
_, _, y_Y_train, y_Y_test = train_test_split(features, target_Y, test_size=0.2, random_state=42)

# Criando e treinando o modelo para coordenada X
model_X = RandomForestRegressor(n_estimators=100)
model_X.fit(X_train, y_X_train)

# Criando e treinando o modelo para coordenada Y
model_Y = RandomForestRegressor(n_estimators=100)
model_Y.fit(X_train, y_Y_train)

# Avaliando os modelos
def evaluate_model(model, X_test, y_test, target_name):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"\nAvaliação para {target_name}:")
    print(f"MSE: {mse:.2f}")
    print(f"R²: {r2:.2f}")
    return y_pred

# Avaliando modelo X
y_X_pred = evaluate_model(model_X, X_test, y_X_test, "Coordenada X")

# Avaliando modelo Y
y_Y_pred = evaluate_model(model_Y, X_test, y_Y_test, "Coordenada Y")
joblib.dump(model_X, 'rf_regressorX.pkl')
joblib.dump(model_Y, 'rf_regressorY.pkl')



'''# Exemplo de previsão
sample_input = np.array([[-0.3, -0.1]])  # Valores fictícios para demonstração
pred_X = model_X.predict(sample_input)
pred_Y = model_Y.predict(sample_input)
print(f"\nExemplo de previsão para input {sample_input[0]}:")
print(f"Coordenada X prevista: {pred_X[0]:.2f}")
print(f"Coordenada Y prevista: {pred_Y[0]:.2f}")'''

In [ ]:
sample_input = np.array([[-0.3, -0.1]])  # Valores fictícios para demonstração
pred_X = model_X.predict(sample_input)
print(pred_X)

In [ ]:
import pandas as pd

# Lendo o arquivo CSV original
df = pd.read_csv('calib_10xPonto/calib9/calib_all_file9.csv', header=None, sep=',')

# Verificando se leu corretamente
if df.shape[1] != 4:
    raise ValueError(f"O arquivo deve ter 4 colunas, mas encontrou {df.shape[1]}")

# Nomeando as colunas
df.columns = ['X', 'Y', 'Label_X', 'Label_Y']

# Criando mapeamento de coordenadas para labels inteiros
coordinate_pairs = list(zip(df['Label_X'], df['Label_Y']))
unique_pairs = sorted(list(set(coordinate_pairs)))  # pares únicos ordenados
label_map = {pair: i+1 for i, pair in enumerate(unique_pairs)}  # mapeia para 1, 2, 3,...

# Criando nova coluna com labels inteiros
df['Label'] = df.apply(lambda row: label_map[(row['Label_X'], row['Label_Y'])], axis=1)

# Selecionando apenas as colunas X, Y e Label (removendo as originais de coordenadas)
new_df = df[['X', 'Y', 'Label']]

# Salvando o novo arquivo CSV
new_filename = 'calib9_label.csv'
new_df.to_csv(new_filename, index=False, header=False)

print(f"Arquivo '{new_filename}' criado com sucesso!")
print("\nMapeamento utilizado:")
for pair, label in label_map.items():
    print(f"Coordenadas {pair} → Label {label}")

In [ ]:
#CLASSIFICAÇÃO
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

# 1. Carregar os dados (assumindo o arquivo com labels inteiras que criamos)
data = pd.read_csv('calib9_label.csv', header=None)
X = data.iloc[:, :2].values  # Primeiras duas colunas (X e Y)
y = data.iloc[:, 2].values   # Última coluna (Label)

# 2. Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 3. Criar e treinar o modelo
model = RandomForestClassifier(n_estimators=30)  # 100 árvores
model.fit(X_train, y_train)

# 4. Avaliar o modelo
accuracy = model.score(X_test, y_test)
print(f"Acurácia do modelo: {accuracy:.2%}")

# 5. Exemplo de previsão
exemplo = [[-0.1, -0.05]]  # Valores de X e Y para predição
predicao = model.predict(exemplo)
print(f"Predição para {exemplo}: classe {predicao[0]}")
joblib.dump(model, 'rf_classifier.pkl')

In [ ]:
y

In [ ]:
data.sort(key=lambda x: (float(x[2]), float(x[3])))

print(data)


In [ ]:
import matplotlib.pyplot as plt

# Definir o número de pontos por subplot
points_per_plot = 30  # Você pode alterar esse valor para o número desejado

# Número de linhas necessárias para plotar todos os subconjuntos em uma grade com 3 colunas
rows = (len(data) + points_per_plot - 1) // points_per_plot // 3 + 1

aux = 0
# Criar uma figura com subplots
fig, axes = plt.subplots(rows, 3, figsize=(15, rows*5))

# Plotando todos os subconjuntos de dados necessários
for index, ax in enumerate(axes.flatten()):
    if index * points_per_plot < len(data):
        # Definir o subconjunto de dados a ser plotado
        info = data[index*points_per_plot:(index+1)*points_per_plot]

        # Extrair os pontos de dados relevantes para plotar
        x_values = [float(point[0]) for point in info]
        y_values = [float(point[1]) for point in info]

        # Plotar os pontos
        ax.scatter(x_values, y_values, marker='o', color='blue')

        # Definir rótulos e título
        ax.set(xlabel='Pitch Values', ylabel='Yaw Values', title=f'{data[aux][2], data[aux][3]}')
        aux += points_per_plot

# Ajustar layout para evitar sobreposição
plt.tight_layout()

# Mostrar o plot
plt.show()




In [ ]:
data_floats = [[ast.literal_eval(sublist) for sublist in group] for group in data]
coord_x = [d[2] for d in data]
coord_y = [d[3] for d in data]
coord_x = list(map(int, map(float, coord_x)))  
coord_y = list(map(int, map(float, coord_y)))  
pitch_yaw = [(d[0], d[1]) for d in data_floats]

print(coord_x)
print(coord_y)
print(pitch_yaw)


In [ ]:
poly_features = []

for py in pitch_yaw:
    py = np.array(py)
    py = py.reshape(1, -1)
    poly_features.append(poly.fit_transform(py))


poly_features = np.array(poly_features)

'''print(len(poly_features))
print(poly_features.shape)
print(poly_features)'''

In [ ]:
#print(poly_features[0][0])
poly_features2 = []
for p in poly_features:
    poly_features2.append(p[0])

In [ ]:
Xx_train, Xx_test, Xy_train, Xy_test = train_test_split(poly_features2, coord_x, test_size=0.2)
Yx_train, Yx_test, Yy_train, Yy_test = train_test_split(poly_features2, coord_y, test_size=0.2)

In [ ]:
automlX = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder="/tmp/autosklearnX_regression_example_tmp",
)
automlY = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder="/tmp/autosklearnY_regression_example_tmp",
)
automlX.fit(Xx_train, Xy_train) #treinando modelo X
automlY.fit(Yx_train, Yy_train) #treinando modelo X


In [ ]:
#SVR
tipo = 'rbf'
svr_modelX = SVR(kernel=tipo, C=1000, gamma=0.1, epsilon=0.4)
svr_modelY = SVR(kernel=tipo, C=1000, gamma=0.1, epsilon=0.4)

svr_modelX.fit(Xx_train, Xy_train)
svr_modelY.fit(Yx_train, Yy_train)

# Fazendo previsões
x_pred = svr_modelX.predict(Xx_test)
y_pred = svr_modelY.predict(Yx_test)

# Avaliação do modelo
mseX = mean_squared_error(Xy_test, x_pred)
mseY = mean_squared_error(Yy_test, y_pred)

r2X = r2_score(Xy_test, y_pred)
r2Y = r2_score(Yy_test, y_pred)


print("Mean Squared Error for X:", mseX)
print("Mean Squared Error for Y:", mseY)
print()
print("R2 for X:", r2X)
print("R2 for Y:", r2Y)



In [ ]:
#RANDOM FOREST
model_x = RandomForestRegressor(max_depth=6)
model_y = RandomForestRegressor(max_depth=6)

model_x.fit(Xx_train, Xy_train)
model_y.fit(Yx_train, Yy_train)

x_pred = model_x.predict(Xx_test)
y_pred = model_y.predict(Yx_test)

mse_x = mean_squared_error(Xy_test, x_pred)
mse_y = mean_squared_error(Yy_test, y_pred)

print()
print("Ground Truth for X: ", Xy_test)
print("Predictions for X:", x_pred)
print()
print("Ground Truth for Y: ", Yy_test)
print("Predictions for Y:", y_pred)
print()

print("Mean Squared Error for X:", mse_x)
print("Mean Squared Error for Y:", mse_y)

In [ ]:
#LINEAR REGRESSION
model_x = LinearRegression()
model_y = LinearRegression()

model_x.fit(Xx_train, Xy_train)
model_y.fit(Yx_train, Yy_train)

x_pred = model_x.predict(Xx_test)
y_pred = model_y.predict(Yx_test)

mse_x = mean_squared_error(Xy_test, x_pred)
mse_y = mean_squared_error(Yy_test, y_pred)

print()
print("Ground Truth for X: ", Xy_test)
print("Predictions for X:", x_pred)
print()
print("Ground Truth for Y: ", Yy_test)
print("Predictions for Y:", y_pred)
print()

print("Mean Squared Error for X:", mse_x)
print("Mean Squared Error for Y:", mse_y)

In [ ]:
#joblib.dump(model_x, 'testeX.pkl')  # Save model_x
#joblib.dump(model_y, 'testeY.pkl')  # Save model_y